In [6]:
from media_impact_monitor.impact import get_impact, ImpactSearch, time_series_regression
from media_impact_monitor.events import get_events, EventSearch
from datetime import date
from media_impact_monitor.trend import get_trend, TrendSearch
import pandas as pd
from great_tables import GT

organizers = ["Fridays for Future", "Extinction Rebellion", "Greenpeace"]
organizers = [organizers[1]]
start_date = date(2022, 1, 1)
end_date = date(2024, 1, 1)
rows = []
for method in ["time_series_regression"]:
    trend = get_trend(
        TrendSearch(
            trend_type="keywords",
            media_source="news_print",
            topic="climate_change",
            start_date=start_date,
            end_date=end_date,
        ),
        as_json=False,
    )["climate activism"].rename("count")
    for organizer in organizers:
        events = get_events(
            EventSearch(
                source="acled",
                organizers=[organizer],
                start_date=start_date,
                end_date=end_date,
            )
        )

        match method:
            case "time_series_regression":
                for lags in [[], [1], [1, 2, 3, 4, 5], [4, 5, 6, 7, 8]]:
                    ts, limitations = time_series_regression(
                        events=events,
                        article_counts=trend,
                        aggregation="daily",
                        lags=lags,
                    )
                    row = dict(
                        method=method,
                        lags=lags,
                    )
                    row[f"ts_{organizer}"] = " ".join(ts["mean"].values.astype(str))
                    rows.append(row)
df = pd.DataFrame(rows)
GT(df).fmt_nanoplot(
    columns=[f"ts_{organizer}" for organizer in organizers][0],
)  # restriction in great_tables, only supports one nanoplot column


GT(_tbl_data=                   method             lags  \
0  time_series_regression               []   
1  time_series_regression              [1]   
2  time_series_regression  [1, 2, 3, 4, 5]   
3  time_series_regression  [4, 5, 6, 7, 8]   

                             ts_Extinction Rebellion  
0  -80.3113626003444 -60.98688660944504 -53.79726...  
1  -47.718162363221765 -29.889902607104105 -23.79...  
2  -20.871819072695757 -3.9902962625004887 1.3066...  
3  13.581461461392024 30.12354950011144 32.256002...  , _body=<great_tables._gt_data.Body object at 0x30281bd00>, _boxhead=Boxhead([ColInfo(var='method', type=<ColInfoTypeEnum.default: 1>, column_label='method', column_align='left', column_width=None), ColInfo(var='lags', type=<ColInfoTypeEnum.default: 1>, column_label='lags', column_align='right', column_width=None), ColInfo(var='ts_Extinction Rebellion', type=<ColInfoTypeEnum.default: 1>, column_label='ts_Extinction Rebellion', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x302fe7b80>, _spanners=Spanners([]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x302ff38b0>, _formats=[<great_tables._gt_data.FormatInfo object at 0x302ff0e50>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), heading_background_color=O